# Reproduce Allen smFISH results with Starfish

The `allen_smFISH.zip` file needed to follow along with this notebook can be downloaded [here]()

This notebook walks through a work flow that reproduces the smFISH result for one field of view using the starfish package. 
It assumes that you have unzipped `allen_smFISH.zip` in the same directory as this notebook. Thus, you should see:

raw/
allen_smFISH.ipynb

In [1]:
import os
from starfish.io import Stack
import numpy as np

from starfish.pipeline.filter.clip import Clip
from starfish.pipeline.filter.bandpass import Bandpass
from starfish.pipeline.filter.gaussian_low_pass import GaussianLowPass

In [2]:
%matplotlib inline
from matplotlib import cm
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection

from scipy import ndimage as ndi
from scipy import stats

from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util, img_as_float)

In [3]:
# package this up same as before
experiment_json = os.path.expanduser('~/google_drive/starfish/data/allen_smFISH/experiment.json')

In [4]:
s = Stack()
s.read(experiment_json)

In [5]:
# The allen's data is uint, but starfish loads it as floats. Cast it to int here, which does not cause any loss of precision. 
s.image._data = s.image._data.astype(np.uint16)
istack = s.image

In [6]:
s_clip = Clip(p_min=10, p_max=100)
s_clip.filter(s)

In [23]:
s_bandpass = Bandpass(lshort=0.5, llong=7, threshold=None, truncate=4)
s_bandpass.filter(s)

In [7]:
from ipywidgets import interact
from starfish.constants import Indices

def vis_z_stack (Stack, h=0, c=0, cmap='gray', rescale=False, figsize=(10, 10)):
    N = Stack.image.shape[Indices.Z]

    data = Stack.image.get_slice({Indices.CH: c, Indices.HYB: h})[0]
    if rescale:
        print("Rescaling ...")
        vmin, vmax = stats.scoreatpercentile(data, (0.5, 99.5))
        data = exposure.rescale_intensity(
            data, 
            in_range=(vmin, vmax), 
            out_range=np.float32
        ).astype(np.float32)

    def show_plane(ax, plane, cmap="gray", title=None):
        ax.imshow(plane, cmap=cmap)
        ax.set_xticks([])
        ax.set_yticks([])

        if title:
            ax.set_title(title)
    
    @interact(plane=(0, N - 1))
    def display_slice(plane=34):
        fig, ax = plt.subplots(figsize=figsize)
        show_plane(ax, data[plane], title="Plane {}".format(plane), cmap=cmap)
        plt.show()

    return display_slice

In [19]:
vis_z_stack(s, c=1, rescale=True);

Rescaling ...


interactive(children=(IntSlider(value=33, description='plane', max=33), Output()), _dom_classes=('widget-interact',))

In [10]:
vis_z_stack(s, c=2, rescale=True);

Rescaling ...


interactive(children=(IntSlider(value=33, description='plane', max=33), Output()), _dom_classes=('widget-interact',))

Image processing function list:

1. clip & floor (at 10th percentile)
  1. `starfish.transform.threshold` or `.clip`
  2. Also called _after_ bandpass. 
2. trackpy bandpass
  1. `filters.bandpass`
4. gaussian filter (over z) 
  1. `starfish.filters.gaussian`. The skimage function supports nd images, so we just need to adjust the object in starfish. 
5. call peak locations (trackpy locate)
  1. `starfish.starfish.spots.crocker-grier` OR `starfish.starfish.spots.gaussian` (second class in this file)

In [1]:
from trackpy import locate

In [2]:
?locate

In [30]:
# ok, so this does not work for singleton barcodes (barcode index should all be 1s)
s.squeeze(bit_map_flag=True)
s.squeeze_map.head(20)

KeyError: 'data'

In [18]:
ls /Users/ajc/google_drive/starfish/data/ISS/fov_001/

1_1st_Cy3 5.TIF     1_2nd_Cy5.TIF       1_4th_Cy3 5.TIF     codebook.csv
1_1st_Cy3.TIF       1_2nd_FITC.TIF      1_4th_Cy3.TIF       experiment.json
1_1st_Cy5.TIF       1_3rd_Cy3 5.TIF     1_4th_Cy5.TIF       hybridization.json
1_1st_FITC.TIF      1_3rd_Cy3.TIF       1_4th_FITC.TIF
1_2nd_Cy3 5.TIF     1_3rd_Cy5.TIF       1_DO_Cy3.TIF
1_2nd_Cy3.TIF       1_3rd_FITC.TIF      1_DO_DAPI.TIF


In [25]:
 # get ISS data, see how that works
s2 = Stack()
s2.read(os.path.expanduser('/Users/ajc/google_drive/starfish/data/ISS/fov_001/experiment.json'))

In [31]:
s.squeeze(bit_map_flag=True)
s.squeeze_map.head(20)

KeyError: 'data'

Goal: 
- Comprehensible format that is understandable to users. 
- Ideally can abstract "features" (spots **and** pixels)
- Does not blow up memory
- Plot features, selectable on the below information

Things one might want to select a spot based on:
- gene
- channel
- hybridization round
- z section
- cell

To construct genes, we need to know codes. What are codes?
- Ordered lists of hybridization rounds and channels
- Merfish: 8 rounds, 2 channels (all hybs, all channels)
- ISS: 4 rounds, 4 channels (all hybs, all channels)
- smFISH: 1 round, 1 channel (all hybs, one channel)
- CODEX: 1 round, 1 channel (one hyb, one channel)

Required Data: 

1. Barcode mapping
2. Num Hybs
3. Num Channels

What's a decoder?
- A decoder is a mapping between (potentially multiple) hybs and channels to genes
- A decoder must be able to map a function across channels (to decide which channel is active) for each hyb
- Features are therefore tracked across hybs/channels
- I need to figure out how MERFISH works again.

So: 
x, y, z, centroids 